# Processing time series

This notebook runs through some sanity checks about the photon flux calculations, comparing them to Bijan's spreadsheet results. This is to ensure that we properly process the input cubes, and gives an idea of how everything is done. Here we take the fiducial example of 47 Uma c.

In [92]:
%pylab inline --no-import-all
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
import os

folder = '../../../code'
print(folder)
if folder not in sys.path: sys.path.append(folder)

Populating the interactive namespace from numpy and matplotlib
../../../code


## Calculate photon fluxes

### Calculate star rates

In [87]:
import glob

# load first filelist to get its shape
fileshape = (45,318,318)


import astropy.units as u
import astropy.constants as c

planet_radius = 1.27*c.R_jup
# ref_star_T=9377*u.K
ref_star_T=5887*u.K
ref_star_Vmag=2.37
target_star_T=5887*u.K
target_star_Vmag=5.03

lamc=770.
BW=0.18
n_ref_star_imgs=30
Nlam = 45
lamlist = lamc*np.linspace(1.-BW/2.,1.+BW/2.,Nlam)*u.nm

#tel_pupil_area=3.650265060424805*u.m**2
tel_pupil_area=1

from tools.inputScene import convert_krist_cube
target_star_cube = convert_krist_cube(fileshape,lamlist,target_star_T,target_star_Vmag,tel_pupil_area)

print("%e" % np.sum(target_star_cube[:,0,0]))

6.416245e+09


In [91]:
# We need to determine the coefficient of proportionality between a blackbody source and the
# actualy flux received (depends on size of the star, distance, etc)
# define Vband
lambda_cent = 550*u.nanometer

# this is the flux density per steradian (specific intensity) you would expect from Vband
flux_bb_F550 = af.blackbody_lambda(lambda_cent, ref_star_T).to(u.Watt/u.m**2/u.um/u.sr)

# this is the actual flux density received in Vband
Vband_zero_pt = (3636*u.Jansky).to(u.Watt/u.m**2/u.Hertz)
Vband_zero_pt *= (c.c/lambda_cent**2)
Vband_zero_pt = Vband_zero_pt.to(u.W/u.m**2/u.um)
flux_star_Vband = Vband_zero_pt*10**(-0.4*0)
E_Vband = (c.c*c.h/lambda_cent).to(u.J)
print("Vband zero point:",Vband_zero_pt)

# the ratio is coefficient we seek; this will multiply a blackbody function to yield flux densities
# at all wavelengths
ratio_star = (flux_star_Vband/flux_bb_F550)
print(ratio_star)

lam = 800*u.nm
flux_bb_band = af.blackbody_lambda(lam, ref_star_T).to(u.Watt/u.m**2/u.um/u.sr)
print("Calculated flux density in band:",ratio_star*flux_bb_band)
print("Ratio of Vband over band spec. intensity: ",flux_bb_F550/flux_bb_band)
E_lam = (c.c*c.h/lam).to(u.J)

lamVIS = 0.8*u.um
VISband_zero_pt = (2550*u.Jansky).to(u.Watt/u.m**2/u.Hertz)
VISband_zero_pt *= (c.c/lamVIS**2)
VISband_zero_pt = VISband_zero_pt.to(u.W/u.m**2/u.um)
print("VISband zero point:",VISband_zero_pt*10**(-0.4*-0.8))



print("Number of photons:")
print((flux_star_Vband/E_Vband).to(1./u.s/u.m**2/u.um))
print((ratio_star*flux_bb_IFS1/E_lam).to(1./u.s/u.m**2/u.um))

('Vband zero point:', <Quantity 3.603455792687603e-08 W / (m2 um)>)
1.2802934685e-15 sr
('Calculated flux density in band:', <Quantity 2.301367765438107e-08 W / (m2 um)>)
('Ratio of Vband over band spec. intensity: ', <Quantity 1.565788765622004>)
('VISband zero point:', <Quantity 2.49563408988201e-08 W / (m2 um)>)
Number of photons:
99771199518.3 1 / (m2 s um)
99060401616.8 1 / (m2 s um)


In [27]:
import astropy.analytic_functions as af

dlam = lamlist[1]-lamlist[0]
high = c.c/(lamlist[0]-dlam/2.)
low = c.c/(lamlist[-1]+dlam/2.)
mid = 0.5*(low+high) # middle frequency
dnu = high-low
E_ph = (c.h*mid).to(u.J) 
print("Us: ",E_ph," Bijan: ",2.58e-19*u.J)

BBlam = af.blackbody_nu(lamlist[Nlam//2], target_star_T).to(u.Watt/u.m**2/u.Hertz/u.sr)
flux = (BBlam*ratio_star*dnu).to(u.W/u.m**2) # this is Watts per m2
photon_flux = flux/E_ph # This is in Photons per second per m2
print(photon_flux.to(1./u.s/u.m**2))
print(100*(lamlist[-1]-lamlist[0])/lamlist[Nlam//2])

('Us: ', <Quantity 2.601843357083208e-19 J>, ' Bijan: ', <Quantity 2.58e-19 J>)
300928913.122 1 / (m2 s)
18.0
